### steering vec 뽑기

In [ ]:
BLOCK_NAMES = [
    "down_blocks_0",
    "down_blocks_1",
    "down_blocks_2",
    "mid_block",
    "up_blocks_1",
    "up_blocks_2",
    "up_blocks_3"
]

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from diffusers import StableDiffusionPipeline
from PIL import Image, ImageDraw, ImageFont
base_path = ""
STEER_DIR     = ""
MODEL_NAME    = "runwayml/stable-diffusion-v1-5"
DEVICE        = "cuda"
OUT_ROOT      = "/real_final"
TEST_CSV      = ""

MEAN_DIR = "MEAN_DIRECTORY"
BASE_ALPHA = 100.0
MAX_ALPHA = 100

# ─── 1) steering vectors load ────────────────────────────────────────────
steer_by_step_block = {}

for step in VECTOR_STEPS:
    steer_by_step_block[step] = {}

    for block_name in BLOCK_NAMES:
        fname = f"step{step}_{block_name}.npy"
        path = os.path.join(STEER_DIR, fname)

        if os.path.exists(path):
            arr = np.load(path).astype(np.float32)
            steer_by_step_block[step][block_name] = torch.from_numpy(arr).to(DEVICE)

print(f"✅ Loaded steering vectors for steps: {VECTOR_STEPS}")

# ─── 2) baseline ────────────────────────────────────────────────────
def generate_baseline(prompt: str, seed: int):
    pipe = StableDiffusionPipeline.from_pretrained(
        MODEL_NAME, torch_dtype=torch.float32,safety_checker=None,feature_extractor=None
    ).to(DEVICE)
    pipe.enable_attention_slicing()
    gen = torch.Generator(device=DEVICE).manual_seed(seed)
    return pipe(prompt,
                num_inference_steps=50,
                guidance_scale=7.5,
                generator=gen).images[0]



✅ Loaded steering vectors for steps: [0, 1, 2, 3, 4, 5, 6, 7, 8]


## multiple

In [ ]:
def generate_steered_dynamic(prompt: str, seed: int):
    pipe = StableDiffusionPipeline.from_pretrained(
        MODEL_NAME, torch_dtype=torch.float32
    ).to(DEVICE)
    pipe.enable_attention_slicing()

    current = {"idx": None}
    def cb(step_idx, timestep, latents):
        current["idx"] = step_idx
        print(f"[CALLBACK] denoise step = {step_idx}")

    # 적용 대상 블록
    TARGET_BLOCKS = [
    "down_blocks_0",
    "down_blocks_1",
    "down_blocks_2",
    "mid_block",
    "up_blocks_1",
    "up_blocks_2",
    "up_blocks_3"
    ]

    # 블록별 기준 거리 계산
    ref_means = {}
    base_ref_dists = {}

    for block_name in TARGET_BLOCKS:
        safe_name = block_name.replace(".", "_")
        ref_mean_path = os.path.join(MEAN_DIR, f"{safe_name}_mean1.npy")  # label 1
        bad_mean_path = os.path.join(MEAN_DIR, f"{safe_name}_mean0.npy")  # label 0

        ref_mean = torch.tensor(np.load(ref_mean_path), dtype=torch.float32).to(DEVICE)
        bad_mean = torch.tensor(np.load(bad_mean_path), dtype=torch.float32).to(DEVICE)
        base_dist = torch.norm(ref_mean - bad_mean, p=2).item()

        ref_means[block_name] = ref_mean
        base_ref_dists[block_name] = base_dist
        print(f"[INFO] {block_name} 기준 거리: {base_dist:.4f}")

      # hook 함수 정의
    def make_hook(layer_name):
      # 블록 이름 파싱
      if "mid_block" in layer_name:
        block_name = "mid_block"
      elif "down_blocks" in layer_name:
          parts = layer_name.split(".")
          block_name = f"{parts[0]}_{parts[1]}"
      elif "up_blocks" in layer_name:
          parts = layer_name.split(".")
          block_name = f"{parts[0]}_{parts[1]}"
      else:
          return lambda *args, **kwargs: None

      # hook 적용 대상 블록 필터링
      if block_name not in TARGET_BLOCKS:
          return lambda *args, **kwargs: None

      def hook(mod, inp, out):
          idx = current["idx"]
          if idx is not None and 1 <= idx <= 10:
              vec_step = idx - 1
              steer = steer_by_step_block.get(vec_step, {}).get(block_name)
              if steer is not None:
                  steer = steer.to(out.device)
                  steer = steer / (steer.norm() + 1e-6)
                  d = steer.shape[0]

                  cur_mean = out.detach().mean(dim=(0, 1))
                  ref_mean = ref_means[block_name]
                  base_dist = base_ref_dists[block_name]

                  # 안전 확인
                  if cur_mean.shape != ref_mean.shape:
                      print(f"[WARN] Shape mismatch in {block_name} at step {idx}")
                      return out

                  error_vec = ref_mean - cur_mean
                  distance = torch.norm(error_vec, p=2).item()

                  # steer 방향과 얼마나 정렬되어 있는지 확인 (cosine similarity)
                  alignment = torch.dot(steer, error_vec) / (steer.norm() * error_vec.norm() + 1e-6)

                  # α 계산 (signed)
                  alpha_ratio = distance / (base_dist + 1e-6)
                  alpha_dyn = BASE_ALPHA * (1 - np.exp(-alpha_ratio))
                  alpha_dyn_signed = alpha_dyn * alignment.item()

                  # Clip
                  alpha_dyn_signed = np.clip(alpha_dyn_signed, -MAX_ALPHA, MAX_ALPHA)

                  # 적용
                  out2 = out + alpha_dyn_signed * steer.view(1, 1, -1)
                  print(f"[{block_name} step {idx}] α={alpha_dyn_signed:.2f} | dist={distance:.2f} | align={alignment.item():.3f}")
                  return out2
          return out
      return hook
    # hook 등록
    handles = []
    for name, mod in pipe.unet.named_modules():
        if "attn2" in name:
            to_q = getattr(mod, "to_q", None)
            if isinstance(to_q, torch.nn.Linear):
                hook_fn = make_hook(name)
                handles.append(to_q.register_forward_hook(hook_fn))

    # 이미지 생성
    gen = torch.Generator(device=DEVICE).manual_seed(seed)
    img = pipe(prompt,
               num_inference_steps=50,
               guidance_scale=7.5,
               generator=gen,
               callback=cb,
               callback_steps=1).images[0]

    for h in handles:
        h.remove()

    return img

# uni

In [ ]:
def generate_steered_dynamic_uni(prompt: str, seed: int):
    pipe = StableDiffusionPipeline.from_pretrained(
        MODEL_NAME, torch_dtype=torch.float32
    ).to(DEVICE)
    pipe.enable_attention_slicing()

    current = {"idx": None}
    def cb(step_idx, timestep, latents):
        current["idx"] = step_idx
        print(f"[CALLBACK] denoise step = {step_idx}")

    TARGET_BLOCKS = [
    "down_blocks_0",
    "down_blocks_1",
    "down_blocks_2",
    "mid_block",
    "up_blocks_1",
    "up_blocks_2",
    "up_blocks_3"
    ]

    ref_means = {}
    base_ref_dists = {}

    for block_name in TARGET_BLOCKS:
        safe_name = block_name.replace(".", "_")
        ref_mean_path = os.path.join(MEAN_DIR, f"{safe_name}_mean1.npy")  # label 1
        bad_mean_path = os.path.join(MEAN_DIR, f"{safe_name}_mean0.npy")  # label 0

        ref_mean = torch.tensor(np.load(ref_mean_path), dtype=torch.float32).to(DEVICE)
        bad_mean = torch.tensor(np.load(bad_mean_path), dtype=torch.float32).to(DEVICE)
        base_dist = torch.norm(ref_mean - bad_mean, p=2).item()

        ref_means[block_name] = ref_mean
        base_ref_dists[block_name] = base_dist
        print(f"[INFO] {block_name} 기준 거리: {base_dist:.4f}")

      # hook 함수 정의
    def make_hook(layer_name):
      # 블록 이름 파싱
      if "mid_block" in layer_name:
        block_name = "mid_block"
      elif "down_blocks" in layer_name:
          parts = layer_name.split(".")
          block_name = f"{parts[0]}_{parts[1]}"
      elif "up_blocks" in layer_name:
          parts = layer_name.split(".")
          block_name = f"{parts[0]}_{parts[1]}"
      else:
          return lambda *args, **kwargs: None

      # hook 적용 대상 블록 필터링
      if block_name not in TARGET_BLOCKS:
          return lambda *args, **kwargs: None

      def hook(mod, inp, out):
          idx = current["idx"]
          if idx is not None and 1 <= idx <= 10:
              vec_step = idx - 1
              steer = steer_by_step_block.get(0, {}).get(block_name)
              if steer is not None:
                  steer = steer.to(out.device)
                  steer = steer / (steer.norm() + 1e-6)
                  d = steer.shape[0]

                  cur_mean = out.detach().mean(dim=(0, 1))
                  ref_mean = ref_means[block_name]
                  base_dist = base_ref_dists[block_name]

                  # 안전 확인
                  if cur_mean.shape != ref_mean.shape:
                      print(f"[WARN] Shape mismatch in {block_name} at step {idx}")
                      return out

                  error_vec = ref_mean - cur_mean
                  distance = torch.norm(error_vec, p=2).item()

                  # steer 방향과 얼마나 정렬되어 있는지 확인 (cosine similarity)
                  alignment = torch.dot(steer, error_vec) / (steer.norm() * error_vec.norm() + 1e-6)

                  # α 계산 (signed)
                  alpha_ratio = distance / (base_dist + 1e-6)
                  alpha_dyn = BASE_ALPHA * (1 - np.exp(-alpha_ratio))
                  alpha_dyn_signed = alpha_dyn * alignment.item()

                  # Clip
                  alpha_dyn_signed = np.clip(alpha_dyn_signed, -MAX_ALPHA, MAX_ALPHA)

                  # 적용
                  out2 = out + alpha_dyn_signed * steer.view(1, 1, -1)
                  print(f"[{block_name} step {idx}] α={alpha_dyn_signed:.2f} | dist={distance:.2f} | align={alignment.item():.3f}")
                  return out2
          return out
      return hook

    handles = []
    for name, mod in pipe.unet.named_modules():
        if "attn2" in name:
            to_q = getattr(mod, "to_q", None)
            if isinstance(to_q, torch.nn.Linear):
                hook_fn = make_hook(name)
                handles.append(to_q.register_forward_hook(hook_fn))

    gen = torch.Generator(device=DEVICE).manual_seed(seed)
    img = pipe(prompt,
               num_inference_steps=50,
               guidance_scale=7.5,
               generator=gen,
               callback=cb,
               callback_steps=1).images[0]

    for h in handles:
        h.remove()

    return img

## save file

In [ ]:
# ─── 4) save utils ────────────────────────────────────────────────────────

from PIL import Image, ImageDraw, ImageFont
import os

def show_and_save(base_img, steer_img1, steer_img2, prompt, seed):
    safe = prompt.replace(" ", "_")
    out_dir = os.path.join(base_path + OUT_ROOT, f"trial9_dynamic_steer_until5_dynamic_alpha_byallblock_first_80{BASE_ALPHA}")
    os.makedirs(out_dir, exist_ok=True)

    bfn = f"{safe}_baseline_{seed}_{BASE_ALPHA}.png"
    s1fn = f"{safe}_steered1_{seed}_{BASE_ALPHA}.png"
    s2fn = f"{safe}_steered2_{seed}_{BASE_ALPHA}.png"
    cfn = f"{safe}_comparison_{seed}_{BASE_ALPHA}.png"

    p_b = os.path.join(out_dir, bfn)
    base_img.save(p_b)
    print("✅", p_b)

    p_s1 = os.path.join(out_dir, s1fn)
    steer_img1.save(p_s1)
    print("✅", p_s1)

    p_s2 = os.path.join(out_dir, s2fn)
    steer_img2.save(p_s2)
    print("✅", p_s2)

    w, h = base_img.size
    th = 40
    canvas = Image.new("RGB", (w * 3, h + th), "white")
    draw = ImageDraw.Draw(canvas)
    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 20)
    except:
        font = ImageFont.load_default()
    draw.text((10, 10), f"[Baseline] {prompt}", fill="black", font=font)
    draw.text((w + 10, 10), f"[Steered1] seed={seed}", fill="black", font=font)
    draw.text((2 * w + 10, 10), f"[Steered2] seed={seed}", fill="black", font=font)

    canvas.paste(base_img, (0, th))
    canvas.paste(steer_img1, (w, th))
    canvas.paste(steer_img2, (2 * w, th))

    p_c = os.path.join(out_dir, cfn)
    canvas.save(p_c)
    print("✅", p_c)

# ─── 5) 실행 ────────────────────────────────────────────────────────────
df = pd.read_csv("TESTDATA_DIRECTORY")


if "label" in df.columns and "prompt" not in df.columns:
    df = df.rename(columns={"label":"prompt"})

for _, row in df.iloc[:].iterrows():
    prompt, seed = row["prompt"], int(row["seed"])
    base_img   = generate_baseline(prompt, seed)
    steer_img  = generate_steered_dynamic(prompt, seed)
    steer_img2 = generate_steered_dynamic_uni(prompt, seed)
    show_and_save(base_img, steer_img,steer_img2, prompt, seed)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/50 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

[INFO] down_blocks_0 기준 거리: 0.0432
[INFO] down_blocks_1 기준 거리: 0.1527
[INFO] down_blocks_2 기준 거리: 0.5639
[INFO] mid_block 기준 거리: 1.2097
[INFO] up_blocks_1 기준 거리: 0.7156
[INFO] up_blocks_2 기준 거리: 0.2204
[INFO] up_blocks_3 기준 거리: 0.0236


...

[CALLBACK] denoise step = 1
[down_blocks_0 step 1] α=11.57 | dist=4.95 | align=0.116
[down_blocks_0 step 1] α=-4.77 | dist=4.85 | align=-0.048
[down_blocks_1 step 1] α=5.52 | dist=11.10 | align=0.055
[down_blocks_1 step 1] α=3.36 | dist=9.75 | align=0.034
[down_blocks_2 step 1] α=13.14 | dist=14.38 | align=0.131
[down_blocks_2 step 1] α=3.80 | dist=15.30 | align=0.038
[mid_block step 1] α=12.08 | dist=8.45 | align=0.121
[up_blocks_1 step 1] α=-4.60 | dist=19.49 | align=-0.046
[up_blocks_1 step 1] α=-0.14 | dist=19.99 | align=-0.001
[up_blocks_1 step 1] α=1.56 | dist=21.27 | align=0.016
[up_blocks_2 step 1] α=6.26 | dist=17.80 | align=0.063
[up_blocks_2 step 1] α=-4.44 | dist=15.15 | align=-0.044
[up_blocks_2 step 1] α=-3.07 | dist=14.85 | align=-0.031
[up_blocks_3 step 1] α=-4.22 | dist=5.89 | align=-0.042
[up_blocks_3 step 1] α=7.77 | dist=7.18 | align=0.078
[up_blocks_3 step 1] α=-2.32 | dist=6.76 | align=-0.023
[CALLBACK] denoise step = 2
[down_blocks_0 step 2] α=12.58 | dist=4.89 |

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

[INFO] down_blocks_0 기준 거리: 0.0432
[INFO] down_blocks_1 기준 거리: 0.1527
[INFO] down_blocks_2 기준 거리: 0.5639
[INFO] mid_block 기준 거리: 1.2097
[INFO] up_blocks_1 기준 거리: 0.7156
[INFO] up_blocks_2 기준 거리: 0.2204
[INFO] up_blocks_3 기준 거리: 0.0236


  0%|          | 0/50 [00:00<?, ?it/s]

[CALLBACK] denoise step = 1
[down_blocks_0 step 1] α=11.57 | dist=4.95 | align=0.116
[down_blocks_0 step 1] α=-4.77 | dist=4.85 | align=-0.048
[down_blocks_1 step 1] α=5.52 | dist=11.10 | align=0.055
[down_blocks_1 step 1] α=3.36 | dist=9.75 | align=0.034
[down_blocks_2 step 1] α=13.14 | dist=14.38 | align=0.131
[down_blocks_2 step 1] α=3.80 | dist=15.30 | align=0.038
[mid_block step 1] α=12.08 | dist=8.45 | align=0.121
[up_blocks_1 step 1] α=-4.60 | dist=19.49 | align=-0.046
[up_blocks_1 step 1] α=-0.14 | dist=19.99 | align=-0.001
[up_blocks_1 step 1] α=1.56 | dist=21.27 | align=0.016
[up_blocks_2 step 1] α=6.26 | dist=17.80 | align=0.063
[up_blocks_2 step 1] α=-4.44 | dist=15.15 | align=-0.044
[up_blocks_2 step 1] α=-3.07 | dist=14.85 | align=-0.031
[up_blocks_3 step 1] α=-4.22 | dist=5.89 | align=-0.042
[up_blocks_3 step 1] α=7.77 | dist=7.18 | align=0.078
[up_blocks_3 step 1] α=-2.32 | dist=6.76 | align=-0.023
[CALLBACK] denoise step = 2
[down_blocks_0 step 2] α=12.41 | dist=4.89 |

## Appendix

### mean vector

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from collections import defaultdict

excel_path = "LABELED_DATA_EXCEL_FILE_PATH"
base_path = "YOUR_PATH"
ttype = "query"
label_col = "label"

for i in range(1):
  step_col = f"step{i}_npy_path"
  df = pd.read_excel(os.path.join(base_path, excel_path))
  df_valid = df[df[step_col].apply(lambda x: os.path.exists(os.path.join(base_path, x)))]

  dims_by_label = {0: defaultdict(list), 1: defaultdict(list)}


  for _, row in df_valid.iterrows():
      path = os.path.join(base_path, row[step_col])
      data = np.load(path, allow_pickle=True).item()
      label = int(row[label_col])

      for block, tensor in data[ttype].items():
          if tensor.ndim != 3:
              continue

          parts = block.split(".")
          if parts[0] in {"down_blocks", "up_blocks"}:
              block_name = ".".join(parts[:2])
          elif parts[0] == "mid_block":
              block_name = "mid_block"
          else:
              continue

          vec = tensor.mean(axis=(0, 1))
          dims_by_label[label][block_name].append(vec)

  steering_output_dir = base_path+"SAVE_PATH"
  os.makedirs(steering_output_dir, exist_ok=True)

  for block_name in sorted(set(dims_by_label[0].keys()) & set(dims_by_label[1].keys())):
      vecs0 = dims_by_label[0][block_name]
      vecs1 = dims_by_label[1][block_name]

      if not vecs0 or not vecs1:
          continue

      vec0_mean = np.mean(np.stack(vecs0), axis=0).astype(np.float32)
      vec1_mean = np.mean(np.stack(vecs1), axis=0).astype(np.float32)
      steering_vector = vec1_mean - vec0_mean

      safe_name = block_name.replace(".", "_").replace("/", "_")
      save_path = steering_output_dir + f"/step{i}_"+f"{safe_name}.npy"
      np.save(save_path, steering_vector)

      print(f"✅ [STEERING] Saved {save_path} | norm: {np.linalg.norm(steering_vector):.4f}")

In [ ]:
mean_output_dir = os.path.join(base_path, "NEW_MEAN_PATH")
os.makedirs(mean_output_dir, exist_ok=True)

for label in [0, 1]:
    for block_name, vec_list in dims_by_label[label].items():
        if not vec_list:
            continue

        mean_vec = np.mean(np.stack(vec_list), axis=0).astype(np.float32)
        safe_name = block_name.replace(".", "_").replace("/", "_")
        save_path = os.path.join(mean_output_dir, f"{safe_name}_mean{label}.npy")
        np.save(save_path, mean_vec)

        print(f"📁 Saved label={label} mean vector for {block_name} → {save_path} | shape: {mean_vec.shape}")